### Requirements

In [1]:
from dataset import*
from utility import*
from training import*
from models import*
from torch.optim import lr_scheduler 

# Head pose DL model from https://github.com/thohemp/6drepnet
from sixdrepnet import SixDRepNet

# Import models
from torchvision import models
from vit_pytorch.twins_svt import TwinsSVT # MEMORIA NON SUFFICIENTE RIPROVARLO
#from vit_pytorch.vit import ViT
from vit_pytorch.ats_vit import ViT
from vit_pytorch import SimpleViT
from vit_pytorch.crossformer import CrossFormer # MEMORIA NON SUFFICIENTE RIPROVARLO
from vit_pytorch.cross_vit import CrossViT

In [2]:
root_project = '/home/anto/University/Driving-Visual-Attention/'

In [3]:
print(f"We have {'' if torch.cuda.is_available() else 'not'} access to a GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.current_device())
    print(torch.cuda.device(0))
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name(0))
print(device)

We have  access to a GPU
0
1
NVIDIA GeForce RTX 3060 Laptop GPU
cuda


In [4]:
seed_everything(42)

##### Initialize pre-trained models for feature extraction

In [5]:
# Initialize face detector and facial landmarks predictor
predictor = dlib.shape_predictor("/home/anto/University/Driving-Visual-Attention/data/shape_predictor_68_face_landmarks.dat")
headpose_extractor = SixDRepNet()
face_detector = dlib.get_frontal_face_detector()
#face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
#face_detector = cv2.FaceDetectorYN.create("/home/anto/University/Driving-Visual-Attention/data/face_detection_yunet_2023mar.onnx",  "", (0, 0))
#face_detector = dlib.cnn_face_detection_model_v1("/home/anto/University/Driving-Visual-Attention/data/mmod_human_face_detector.dat")
#from facenet_pytorch import MTCNN
#face_detector = MTCNN(keep_all=False, device=device, post_process=False)

### Data Loader and Visualization

##### Files where to write the paths and labels

In [6]:
percentage = 10
save_train_file = root_project + 'save/save_train' + str(percentage)
save_val_file = root_project + 'save/save_val' + str(percentage)
save_test_file = root_project + 'save/save_test' + str(percentage)

##### Train Loader

In [7]:
#%prun -s cumulative 
train_dataset_classloader = DataLoaderVisualizer(root_project, save_train_file, percentage, predictor, face_detector, headpose_extractor, 'train')

Building path structure
Loading data


  0%|          | 0/16 [00:08<?, ?it/s]


KeyboardInterrupt: 

##### Validtion Loader

In [ ]:
val_dataset_classloader = DataLoaderVisualizer(root_project,save_val_file,percentage,predictor, face_detector, headpose_extractor,'val')

##### Test Loader

In [ ]:
#test_dataset_classloader = DataLoaderVisualizer(root_project,save_test_file,percentage,predictor, face_detector, headpose_extractor,'test')

##### Visualization

In [ ]:
train_dataset_classloader.visualize_dataset()

In [ ]:
val_dataset_classloader.visualize_dataset()

In [ ]:
#test_dataset_classloader.visualize_dataset()

### Pytorch Dataset 

In [ ]:
# Choose size of the eyes
dim = (128,224)
# mean and std of images, calculated in advance
mean = (0.4570, 0.4422, 0.3900)
std = (0.2376, 0.2295, 0.2261)

my_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(dim, antialias=True),
    transforms.Normalize(mean=mean, std=mean, inplace=True)
])

In [ ]:
train_dataset = DGAZEDataset('train','save/save_train'+str(percentage),my_transforms)
print(f'Train dataset len is {len(train_dataset)}')

In [ ]:
# Print an example of the dataset for correct visualization
img_np = train_dataset[30][0].permute(1, 2, 0).numpy()
plt.imshow(img_np)
plt.axis('off')
plt.show()

In [ ]:
val_dataset = DGAZEDataset('val','save/save_val'+str(percentage),my_transforms)
print(f'Val dataset len is {len(val_dataset)}')

In [ ]:
#test_dataset = DGAZEDataset('test','save/save_test'+str(percentage),my_transforms)
#print(f'Test dataset len is {len(test_dataset)}')

### Vision Transformer Model

##### Hyerparameters

In [ ]:
EPOCHS = 50
BATCH_SIZE = 16
'''
IMAGE_SIZE = 768
HEADS = 8
DROPOUT = 0.1
PATCH_SIZE = 32
DIM = 1024
MLP_DIM= 1024
DEPTH = 4
'''
THRESHOLD = 200
pre_trained = False

In [ ]:
'''
model = ViT(
    image_size = IMAGE_SIZE,
    patch_size = PATCH_SIZE,
    num_classes = 2,
    channels= 3,
    pool='mean',
    dim = DIM,
    depth = DEPTH,
    heads = HEADS,
    mlp_dim = MLP_DIM,
    dropout = DROPOUT,
    emb_dropout = 0.1
)

model = SimpleViT(
    image_size = IMAGE_SIZE,
    patch_size = PATCH_SIZE,
    num_classes = 2,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)

model = CrossViT(
    image_size = IMAGE_SIZE,
    num_classes = 2,
    depth = 2,               # number of multi-scale encoding blocks
    sm_dim = 192,            # high res dimension
    sm_patch_size = 16,      # high res patch size (should be smaller than lg_patch_size)
    sm_enc_depth = 1,        # high res depth
    sm_enc_heads = 4,        # high res heads
    sm_enc_mlp_dim = 1024,   # high res feedforward dimension
    lg_dim = 384,            # low res dimension
    lg_patch_size = 64,      # low res patch size
    lg_enc_depth = 3,        # low res depth
    lg_enc_heads = 8,        # low res heads
    lg_enc_mlp_dim = 1024,   # low res feedforward dimensions
    cross_attn_depth = 1,    # cross attention rounds
    cross_attn_heads = 4,    # cross attention heads
    dropout = 0.1,
    emb_dropout = 0.1
)

model = ViT(
    image_size = IMAGE_SIZE,
    patch_size = PATCH_SIZE,
    num_classes = 2,
    dim = 1024,
    depth = 6,
    max_tokens_per_depth = (256, 128, 64, 32, 16, 8), # a tuple that denotes the maximum number of tokens that any given layer should have. if the layer has greater than this amount, it will undergo adaptive token sampling
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

class GazeHeadResNet(nn.Module):
    def __init__(self):
        super(GazeHeadResNet, self).__init__()
        self.resnet34 = models.resnet34(pretrained=True)
        #self.resnet34.avgpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
        self.resnet34.fc = nn.Linear(in_features=512, out_features=4, bias=True)

    def forward(self, X):
        h = self.resnet34(X)
        gaze_hat = h[:, :2]
        head_hat = h[:, 2:]
        return gaze_hat
    
model = GazeHeadResNet()
'''
model = GazeCNN()
model.to(device)

##### Criterion and Optimizer

In [ ]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.95))
scheduler = lr_scheduler.StepLR(optimizer, step_size=50000, gamma=0.1)

##### Dataloader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Training 

In [ ]:
if pre_trained:
    ckpt_path = ''
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
wandb.login()

In [ ]:
wandb.init(project="GazeViT", name="Modello Nostro lr = 0.01 scheduler")

In [ ]:
if pre_trained:
    start_epoch = checkpoint['epoch']
    EPOCHS = start_epoch + EPOCHS
else:
    start_epoch = 0
    EPOCHS = EPOCHS

for epoch in range(start_epoch, EPOCHS):
    # Training
    train_loss = train_epoch(model, train_loader, criterion, scheduler, optimizer, device, epoch)
    wandb.log({"epoch": epoch + 1,"train_loss": train_loss})

    # Validation
    val_loss, val_accuracy = validate(model, val_loader, THRESHOLD, criterion, device, epoch)
    wandb.log({"epoch": epoch + 1,"val_loss": val_loss})
    wandb.log({"epoch": epoch + 1,"val_accuracy": val_accuracy})

    log_image(val_loader, model, device)

# Finish the WandB run
wandb.finish()

In [ ]:
save_dict = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(save_dict, root_project + 'save/AdaptiveToken_weights_epochs' + str(EPOCHS)+ '.pth')

### Test